I found a bunch of data from https://www.bts.gov/ and https://www.faa.gov/airports available for download. 

My goal with these data sets is to create variables I can merge together for my own data set. 

First thing: read in and preview each file...

All the data files are .xlsx or .xls (except for the one pdf?) and can be found at: /Users/lucystorts/Desktop/BYU/Stat386/386Project/PotentialData

In [1]:
import pandas as pd
import numpy as np
import requests
import re
import urllib.parse
import matplotlib.pyplot as plt             
import seaborn as sns
import json
from datetime import datetime

In [74]:
partialpath = '/Users/lucystorts/Desktop/BYU/Stat386/386Project/PotentialData/'

In [22]:
# !pip install xlrd

# !conda install -c anaconda xlrd --yes


Notes on data: 

**rankings** 
- U.S. Airports, ranked by 2022 Systemwide* Scheduled Enplanements Passenger numbers in millions (000,000)

**OriginatingPassengers**
- U.S. Airports ranked by 2022 Originating Domestic Passengers  
- Source: Bureau of Transportation Statistics, Origin & Destination Survey
- DB1B Ticket, Based on 10 Percent Ticket Sample
- O&D numbers are not comparable to T-100 Market Enplanement numbers

**airportArrivalsOnTimeCurrentMonthly**
- Table 3
- Ranking of Major Airport On-Time Arrival Performance in for Sept 22 - Aug 23					
- Based on Marketing Carrier Performance, (Percent On-Time)					

**airportArrivalsOnTimeHistorical**
- Table 4
- Ranking of Major Airport On-Time Arrival Performance Year-to-date through December
- tables for 2004 - 2022						
- Based on Marketing Carrier Performance, (Percent On-Time)		

**airportDepartureOnTimeCurrentMonthly**
- Table 5
- Ranking of Major Airport On-Time Departure Performance for Sept 22 - Aug 23						
- Based on Marketing Carrier Performance, (Percent On-Time)								

In [76]:
rankings = pd.read_excel(partialpath+'AirportRankings2022.xlsx')
originatingPassengers = pd.read_excel(partialpath+'OriginatingPassengers.xlsx')
airportLOCID = pd.read_excel(partialpath+'NPIAS-Report-2019-2023-Appendix-A.xlsx')
airportPFC = pd.read_excel(partialpath+'pfc_airports_20231031_0.xlsx')
airportPassengersBoarded = pd.read_excel(partialpath+'table_01_44_042523.xlsx')
airportGrants = pd.read_excel(partialpath+'airportGrants.xlsx')

/Users/lucystorts/opt/anaconda3/envs/stat386/lib/python3.10/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [70]:
onTimePath = partialpath+'AnnualAirlineOn-TimeRankings2003-2020.xlsx'
excelFile = pd.ExcelFile(onTimePath)
sheetNames = excelFile.sheet_names
airlineArrivalsOnTime = {}
for sheetName in sheetNames:
    airlineArrivalsOnTime[sheetName] = excelFile.parse(sheetName)

# use .keys() to see all tables names!
airlineArrivalsOnTime['2003'].head(3)

,Unnamed: 0,Carrier,On-Time Arrival Pct.
0,1,Hawaiian,90.77
1,2,SkyWest,87.59
2,3,Southwest,86.30


In [71]:
onTimePath = partialpath+'Table3-RankingMajorAirportOn-TimeArrivalPerformanceAugust2023.xlsx'
excelFile = pd.ExcelFile(onTimePath)
sheetNames = excelFile.sheet_names
airportArrivalsOnTimeCurrentMonthly = {}
for sheetName in sheetNames:
    airportArrivalsOnTimeCurrentMonthly[sheetName] = excelFile.parse(sheetName)

# use .keys() to see all tables names!
airportArrivalsOnTimeCurrentMonthly['Aug23'].head(3)

,Rank,2022-08-01 00:00:00,%,Rank.1,2023-08-01 00:00:00,%.1,Unnamed: 6,
0,1,"Salt Lake City, UT (SLC)",84.62,1.0,"Salt Lake City, UT (SLC)",83.87,NaN,NaN
1,2,"Seattle, WA (SEA)",84.40,2.0,"Minneapolis/St. Paul, MN (MSP)",83.74,NaN,NaN
2,3,"Minneapolis/St. Paul, MN (MSP)",83.07,3.0,"Washington, DC (IAD)",82.91,NaN,NaN


In [72]:
onTimePath = partialpath+'Table4RankingMajorAirportOn-TimeArrivalPerformanceYear-to-dateDecember2003-Dec2022.xlsx'
excelFile = pd.ExcelFile(onTimePath)
sheetNames = excelFile.sheet_names
airportArrivalsOnTimeHistorical = {}
for sheetName in sheetNames:
    airportArrivalsOnTimeHistorical[sheetName] = excelFile.parse(sheetName)

# use .keys() to see all tables names!
airportArrivalsOnTimeCurrentMonthly['Aug23'].head(3)

,Rank,2022-08-01 00:00:00,%,Rank.1,2023-08-01 00:00:00,%.1,Unnamed: 6,
0,1,"Salt Lake City, UT (SLC)",84.62,1.0,"Salt Lake City, UT (SLC)",83.87,NaN,NaN
1,2,"Seattle, WA (SEA)",84.40,2.0,"Minneapolis/St. Paul, MN (MSP)",83.74,NaN,NaN
2,3,"Minneapolis/St. Paul, MN (MSP)",83.07,3.0,"Washington, DC (IAD)",82.91,NaN,NaN


In [73]:
onTimePath = partialpath+'Table5-RankingMajorAirportOn-TimeDeparturePerformanceAugust2023.xlsx'
excelFile = pd.ExcelFile(onTimePath)
sheetNames = excelFile.sheet_names
airportDepartureOnTimeCurrentMonthly = {}
for sheetName in sheetNames:
    airportDepartureOnTimeCurrentMonthly[sheetName] = excelFile.parse(sheetName)

airportDepartureOnTimeCurrentMonthly['Aug23'].head(3)

,Rank,2022-08-01 00:00:00,%,Rank.1,2023-08-01 00:00:00,%.1
0,1,"Salt Lake City, UT (SLC)",83.78,1.0,"Salt Lake City, UT (SLC)",82.55
1,2,"San Francisco, CA (SFO)",82.89,2.0,"Minneapolis/St. Paul, MN (MSP)",81.81
2,3,"Detroit, MI (DTW)",82.86,3.0,"Seattle, WA (SEA)",81.65


In [45]:
rankings.head(3)

,2022 Rank,Airport,2022 Enplaned Passengers,2021 Rank,2021 Enplaned Passengers,Pct. Change 2021-2022
0,1,"Atlanta, GA: Hartsfield-Jackson Atlanta Intern...",45.3721,1.0,36.6658,23.745016
1,2,"Dallas/Fort Worth, TX: Dallas/Fort Worth Inter...",35.3294,2.0,30.0006,17.762311
2,3,"Denver, CO: Denver International",33.7499,3.0,28.6285,17.889166


In [48]:
originatingPassengers.head(3)

,Unnamed: 0,Airport,Code,Originating Domestic Passengers
0,1.0,"Los Angeles, CA: Los Angeles International",LAX,1324394
1,2.0,"Denver, CO: Denver International",DEN,1138192
2,3.0,"Atlanta, GA: Hartsfield-Jackson Atlanta Intern...",ATL,1088364


In [59]:
airportLOCID.head(3)

,State,City,Aiport,LOCID,Ownership,Hub,Role,Current,Year 5,Enplaned,Based,2019-2023 Dev Estimate
0,AK,Adak Island,Adak,ADK,PU,NaN,Basic,GA,GA,2151,0,16166667
1,AK,Akhiok,Akhiok,AKK,PU,NaN,Basic,GA,GA,399,0,8161404
2,AK,Akiachak,Akiachak,Z13,NaN,NaN,Local,CS,GA,3045,0,0


In [58]:
airportPFC.head(3)

,Associated City,State,Airport Name,LOC ID,Hub size,Level,Start Date,Expiration Date,Total PFC Approved (by location)
0,Fairbanks,AK,Fairbanks International,FAI,S,3.0,2000-10-01,2004-04-01,NaN
1,Fairbanks,AK,Fairbanks International,FAI,S,4.5,2004-04-01,2026-10-01,38413252.0
2,Juneau,AK,Juneau International,JNU,N,3.0,1998-10-01,2001-02-01,NaN


In [63]:
airportPassengersBoarded.head(3)

,Airport,Code,Rank,Total enplaned passengers 2012,Rank.1,Total enplaned passengers 2021,Rank.2,Total enplaned passengers 2022,Percent change 2012-2022,Percent change 2021-2022
0,"Atlanta, GA (Hartsfield-Jackson Atlanta Intern...",ATL,1.0,45799067.0,1.0,36686787.0,1.0,45395960.0,-0.880164,23.739263
1,"Dallas/Fort Worth, TX (Dallas/Fort Worth Inter...",DFW,4.0,27100907.0,2.0,30011009.0,2.0,35345122.0,30.420439,17.773854
2,"Denver, CO (Denver International)",DEN,5.0,25242113.0,3.0,28648845.0,3.0,33773762.0,33.799266,17.888739


In [77]:
airportGrants.head(3)

,State,City,Airport,LOCID,Project Description,Total Amount
0,NaN,NaN,NaN,NaN,NaN,NaN
1,AK,Girdwood,Girdwood,AQY,Seal Apron Pavement Surface/Pavement Joints,504078.0
2,NaN,NaN,NaN,NaN,NaN,NaN
